In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mean_exp_fitness(pop_dist, fitness_list):
    return np.sum(pop_dist*np.exp(fitness_list))/np.sum(pop_dist)

def wright_fisher_probabilities(pop_dist, fitness_list):
    return np.exp(fitness_list)/mean_exp_fitness(pop_dist, fitness_list)*pop_dist/np.sum(pop_dist)

def wright_fisher_fitness_update(pop_dist, fitness_list):
    probabilities = wright_fisher_probabilities(pop_dist, fitness_list)
    shape = probabilities.shape
    probabilities.shape = shape[0]*shape[1]
    new = np.random.multinomial(np.sum(pop_dist), probabilities)
    new.shape = shape
    return new

In [3]:
fitnesses = np.transpose(np.atleast_2d(np.linspace(.05,-.05,11)))
print(fitnesses)

[[ 0.05]
 [ 0.04]
 [ 0.03]
 [ 0.02]
 [ 0.01]
 [ 0.  ]
 [-0.01]
 [-0.02]
 [-0.03]
 [-0.04]
 [-0.05]]


In [4]:
pop_dist = 10**7*np.ones((11,5)).astype('int32')
print(pop_dist)
print(np.sum(pop_dist))

[[10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]]
550000000


In [5]:
probabilities=wright_fisher_probabilities(pop_dist, fitnesses)
print(probabilities)
print(np.sum(probabilities))

[[ 0.01910447  0.01910447  0.01910447  0.01910447  0.01910447]
 [ 0.01891437  0.01891437  0.01891437  0.01891437  0.01891437]
 [ 0.01872617  0.01872617  0.01872617  0.01872617  0.01872617]
 [ 0.01853984  0.01853984  0.01853984  0.01853984  0.01853984]
 [ 0.01835537  0.01835537  0.01835537  0.01835537  0.01835537]
 [ 0.01817273  0.01817273  0.01817273  0.01817273  0.01817273]
 [ 0.01799191  0.01799191  0.01799191  0.01799191  0.01799191]
 [ 0.01781289  0.01781289  0.01781289  0.01781289  0.01781289]
 [ 0.01763565  0.01763565  0.01763565  0.01763565  0.01763565]
 [ 0.01746017  0.01746017  0.01746017  0.01746017  0.01746017]
 [ 0.01728644  0.01728644  0.01728644  0.01728644  0.01728644]]
1.0


In [6]:
wag = pop_dist
for i in range(10):
    wag = wright_fisher_fitness_update(wag, fitnesses)
print(wag)

[[15683714 15691462 15674875 15673759 15708404]
 [14186778 14203015 14211441 14179023 14209865]
 [12857987 12870420 12851985 12836150 12836509]
 [11634082 11629136 11621841 11609877 11628908]
 [10516626 10536440 10505972 10507509 10516745]
 [ 9522156  9519415  9517860  9510241  9515229]
 [ 8607414  8629216  8612593  8611801  8613255]
 [ 7788742  7789015  7788752  7790974  7779019]
 [ 7036767  7048769  7047905  7055402  7070090]
 [ 6387284  6372573  6371666  6388683  6370215]
 [ 5775295  5770091  5777680  5773281  5776094]]


In [7]:
np.sum(wag)

550000000

In [8]:
def array_multinomial(N_array, Pis_array, checks=True):
    """
    Draw from multinomial distribution P(x1, x2,..., xi; N, p1, p2, ... pi) array wise where N's are from N_array
    and their respective pi's are from the Pis array with matching trailing indices and the leading index is the
    value of i.
    
    Return an array of xis in the same shape as the array Pis_array.
    
    The last subarray along the leading axis of Pis_array is ignored if checks is False because it is assumed to be
    equal to 1 minus the rest of that axis since probability is conserved. If checks is True then we check to make
    sure all sums over pi's are equal to 1 as required.
    """

    if checks is True:
        if not np.all(Pis_array>=0):
            raise ValueError('All probabilities must be 0 or positive.')
        total_probability = np.sum(Pis_array, axis=0)
        if not np.all(np.isclose(total_probability,1., rtol=0, atol=1e-15)):
            raise ValueError('The total probability parameters of a multinomial distribution must sum to 1.')

    if Pis_array.shape[1:] != N_array.shape:
        raise AttributeError('Pis_array must be the shape of N_array plus one additional axis in the lead')

    Xis_array = np.zeros_like(Pis_array, dtype='int32')
    N_remain_array = np.copy(N_array)
    prob_remain = np.ones_like(N_array, dtype='float64')
    for i in range(Pis_array.shape[0]-1):
        Xis_array[i,...]=np.random.binomial(N_remain_array, Pis_array[i,...]/prob_remain)
        N_remain_array -= Xis_array[i,...]
        prob_remain -= Pis_array[i,...]
    Xis_array[-1,...]=N_remain_array
    return Xis_array

In [9]:
N_array = np.ones((30,30),dtype='int32')*1000
print(N_array)
p_array = np.zeros((5,30,30))
p_array[0,...]=np.arange(900).reshape((30,30))*.0003
p_array[1,...]= .3
p_array[2,...]= .2
p_array[3,...]= .05
p_array[4,...]= 1-p_array[0,...]-p_array[1,...]-p_array[2,...]-p_array[3,...]
print(p_array)

[[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1

In [10]:
multidraw = array_multinomial(N_array, p_array)
print(multidraw[:,0:10,0:10])

[[[  0   1   3   0   0   4   3   2   0   1]
  [  7  10  10   5  10  10  14  12  13   4]
  [ 12  20  18  14  22  25  17  16  23  19]
  [ 28  35  38  33  30  13  35  33  34  36]
  [ 25  37  31  34  41  32  46  42  41  40]
  [ 43  42  50  54  42  36  39  49  36  41]
  [ 66  51  54  56  53  49  54  51  46  66]
  [ 71  46  74  72  81  73  61  67  70  60]
  [ 70  60  81  76  72  75  77  67  70  89]
  [ 88  92  74  88  78  98  79  97  95  78]]

 [[284 307 298 316 314 321 307 277 286 299]
  [289 296 300 299 306 305 309 308 293 314]
  [295 293 306 288 306 304 293 278 289 303]
  [309 279 287 313 295 297 295 324 320 312]
  [288 302 282 307 293 306 296 337 303 290]
  [300 291 277 274 316 309 296 307 294 323]
  [289 300 294 300 308 325 291 318 327 306]
  [289 310 292 311 283 289 302 297 309 319]
  [306 290 292 303 293 314 275 288 304 306]
  [298 293 300 291 311 254 316 303 301 288]]

 [[197 218 219 195 186 182 194 198 197 228]
  [209 192 190 217 182 209 197 204 182 181]
  [180 204 207 197 192 202 2

In [11]:
np.sum(multidraw, axis=0)

array([[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
       [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
       [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
       [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
       [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 

In [12]:
%timeit array_multinomial(N_array, p_array)

The slowest run took 5.15 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 657 µs per loop


In [13]:
def multibase_increment(tup, tuple_bounds):
    new_tup = ()
    increment = 1
    for i in range(-1,-len(tup)-1,-1):
        num = tup[i]
        bound = tuple_bounds[i]
        if num + increment < bound:
            new_tup = ((num + increment),) + new_tup
            if increment == 1:
                increment = increment - 1
        else:
            new_tup = (0,) + new_tup
    return new_tup

#saves time because it avoid allocations
def multibase_increment_fast(lis, list_bounds):
    i = -1
    while i >= -len(lis):
        if lis[i] + 1 < list_bounds[i]:
            lis[i] = lis[i] + 1
            break
        else:
            lis[i]=0
            i = i - 1
        
def int_to_indices(integer, dimensions):
    dimensions = np.array(dimensions)
    if integer > np.product(dimensions):
        raise ValueError('The number of entries in an array of these dimensions is less than integer')
    remaining = integer
    indices = () #[0]*dimensions.size
    for i in range(-1, -dimensions.size-1, -1):
        base = dimensions[i]
        digit = remaining % base
        remaining = (remaining - digit) // base
        indices = (digit,) + indices #indices[i] = digit
    return indices
    

In [14]:
int_to_indices(12, (3,3,3,3))

(0, 1, 1, 0)

In [15]:
def subarray_view(array, axes, indices, checks=True):
    '''Return view of subarray of input array with given axes fixed at corresponding indices'''
    if checks:
        # Coerce the inputs into flat numpy arrays to allow for easy handling of a variety of input types
        axes = np.atleast_1d(np.array(axes)).flatten()
        indices = np.atleast_1d(np.array(indices)).flatten()
        # Check against accessing nonexistent axes
        if np.max(axes) >= array.ndim or np.min(axes)< -array.ndim:
            raise IndexError('too many indices for array')
        # negative axes handled like negative indices in numpy arrays
        for index, element in enumerate(axes):
            if element < 0:
                axes[index] = element + array.ndim
        if axes.shape != indices.shape:
            raise ValueError('axes and indices must have matching shapes or both be integers')

    colon = slice(None, None, None)
    multislice = ()
    for i in range(array.ndim):
        if i in axes:
            multislice = multislice + (indices[np.where(axes==i)[0][0]],)
        else:
            multislice = multislice + (colon,)
    return array[multislice]

def subrange_view(array, starts, ends, steps=None, checks=True):
    if checks:
        # Coerce the inputs into flat numpy arrays to allow for easy handling of a variety of input types
        starts = np.atleast_1d(np.array(starts)).flatten()
        ends = np.atleast_1d(np.array(ends)).flatten()
        if steps is not None:
            steps = np.atleast_1d(np.array(steps)).flatten()
        # Check number of array axes matches up with starts and ends
        if (array.ndim != starts.size) or (array.ndim != ends.size):
            raise ValueError('the size of starts and ends must equal the number of array dimensions')
    multislice = ()
    # If steps is None, default to step size of 1
    if steps is None:
        for i in range(array.ndim):
            multislice = multislice + (slice(starts[i], ends[i], 1),)
    else:
        for i in range(array.ndim):
            multislice = multislice + (slice(starts[i], ends[i], steps[i]),)
    return array[multislice]
    

def stenciled_sum(big_array, axes, stencil, checks=True):
    if checks:
        axes = np.atleast_1d(np.array(axes)).flatten()
        # Check against accessing nonexistent axes
        if np.max(axes) >= big_array.ndim or np.min(axes)< -big_array.ndim:
            raise IndexError('too many indices for array')
        # negative axes handled like negative indices in numpy arrays
        for index, element in enumerate(axes):
            if element < 0:
                axes[index] = element + big_array.ndim
        # Check that stencil shape is correct
        correct_stencil_shape = np.hstack( [np.array(big_array.shape)[axes],
                                            np.array(big_array.ndim - len(axes))] )
        #if we're summing across every axis, then just call np.sum 
        #(this avoids complicating rest of code for simple special case)
        if correct_stencil_shape[-1] == 0:
            return np.sum(big_array)
        if not np.all(np.array(stencil.shape) == correct_stencil_shape):
            raise ValueError('The shape of the stencil must match the big array and axes appropriately')

    # make array of the axes we're preserving
    not_axes = [i for i in range(big_array.ndim) if i not in axes] #np.setdiff1d(np.arange(big_array.ndim),axes)
    #tuple of all but last stencil axis
    ablsa = tuple(range(stencil.ndim-1))
    return_array_shape = np.array(big_array.shape)[not_axes] + np.amax(stencil, axis=ablsa) - np.amin(stencil, axis=ablsa)
    
    #left zero the stencil
    stencil = stencil - np.amin(stencil, axis=ablsa)
    
    #perform the stenciled summation
    return_array = np.zeros(return_array_shape, dtype = big_array.dtype)
    iter_bounds = stencil.shape[:-1]
    final_loop = np.product(np.array(iter_bounds))
    index = np.zeros(len(iter_bounds), dtype='int_')
    subarray_size = np.array(subarray_view(big_array, axes, index, checks=checks).shape)
    for i in range(final_loop):
        starts = stencil[tuple(index)]
        ends = stencil[tuple(index)] + subarray_size
        chunk_to_increase = subrange_view(return_array, starts, ends, checks=checks)
        chunk_to_increase[:] += subarray_view(big_array, axes, index, checks=checks)
        multibase_increment_fast(index, iter_bounds)
    return return_array

In [16]:
wurg = np.arange(8).reshape((2,2,2))
print(wurg)

[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]


In [17]:
stencil = np.eye(2, dtype='int_')

In [18]:
stenciled_sum(wurg, 0, stencil)

array([[0, 4, 5],
       [0, 7, 7],
       [2, 3, 0]])

In [19]:
def unstenciled_sum(big_array, axes, checks=True):
    '''Sum big_array across axes.
    
    Should return a result equivalent to calling np.sum(big_array, axis = tuple(axes)).'''
    correct_stencil_shape = np.hstack( [np.array(big_array.shape)[axes],
                                        np.array(big_array.ndim - len(axes))] )
    stencil = np.zeros(correct_stencil_shape, dtype='int_')
    return stenciled_sum(big_array, axes, stencil, checks)

def test_equality_summation(big_array, axes, verbose=False):
    '''Test equality of unstenciled sum and np.sum with same arguments.'''
    if verbose:
        print('summation using unstenciled_sum is', unstenciled_sum(big_array,axes))
        print('summation using np.sum is', np.sum(big_array, axis = tuple(axes)))
    return np.all(np.sum(big_array, axis = tuple(axes)) == unstenciled_sum(big_array, axes))

def random_eq_test(verbose=False):
    ndim = np.random.randint(1, 7)
    shape = np.random.randint(1, 7, size=ndim)
    n_axes_sum = np.random.randint(1, ndim+1)
    axes_sum = np.random.choice(np.arange(n_axes_sum), size=n_axes_sum, replace=False)
    array = np.random.randint(100, size=shape)
    if verbose:
        print('shape is', shape)
        print('axes summed over is', axes_sum)
        print('array is', array)
    return test_equality_summation(array, axes_sum, verbose)

In [20]:
random_eq_test(True)

shape is [5 4 3 6 2]
axes summed over is [0 1]
array is [[[[[26 85]
    [62 72]
    [39 42]
    [97 49]
    [20 27]
    [51 26]]

   [[58 67]
    [59 98]
    [80 79]
    [84 12]
    [78  4]
    [18 70]]

   [[50 84]
    [77 94]
    [57 22]
    [78 12]
    [96 77]
    [20 12]]]


  [[[52 18]
    [60 54]
    [32 17]
    [47  3]
    [29 53]
    [52 15]]

   [[67 68]
    [87  8]
    [ 6 12]
    [25 60]
    [ 0 93]
    [18 13]]

   [[28 69]
    [49  4]
    [74 53]
    [95 43]
    [58 40]
    [36 72]]]


  [[[84 45]
    [72 13]
    [96 85]
    [95 38]
    [54 19]
    [83 47]]

   [[32 98]
    [82 46]
    [92 24]
    [67 44]
    [53 52]
    [21 23]]

   [[96 30]
    [68 35]
    [50 77]
    [84 64]
    [72 38]
    [95 83]]]


  [[[60 37]
    [97 33]
    [34 20]
    [95 70]
    [70 95]
    [40 59]]

   [[ 9 39]
    [36 35]
    [96  5]
    [64  2]
    [35  2]
    [61 41]]

   [[27  7]
    [87 94]
    [72 43]
    [14 62]
    [66 43]
    [10 79]]]]



 [[[[19 72]
    [14 82]
    [49 66]
    [45 38

True

In [21]:
test_number = 10**4
tests = np.zeros(shape=(test_number), dtype=bool)
for i in range (len(tests)):
    tests[i] = random_eq_test()
print(sum(tests))

10000


In [22]:
import cProfile

In [23]:
cProfile.run('[unstenciled_sum(np.random.randint(100, size=(5,200,200)), axes=np.array([0]), checks=False) for i in range(1000)]', sort = 'cumtime')

         70004 function calls in 1.740 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.740    1.740 {built-in method builtins.exec}
        1    0.012    0.012    1.740    1.740 <string>:1(<module>)
        1    0.003    0.003    1.728    1.728 <string>:1(<listcomp>)
     1000    1.406    0.001    1.406    0.001 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.007    0.000    0.316    0.000 <ipython-input-19-cc9e41348ac7>:1(unstenciled_sum)
     1000    0.156    0.000    0.294    0.000 <ipython-input-15-15f28e7aa83f>:47(stenciled_sum)
     3000    0.057    0.000    0.057    0.000 {built-in method numpy.core.multiarray.zeros}
     6000    0.034    0.000    0.037    0.000 <ipython-input-15-15f28e7aa83f>:1(subarray_view)
     4000    0.013    0.000    0.013    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1000    0.001    0.000    0.011    0.000 shape_base.py:236(hsta

In [24]:
cProfile.run('[np.sum(np.random.randint(100, size=(5,200,200)), axis=(0)) for i in range(1000)]', sort = 'cumtime')

         5004 function calls in 1.569 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.569    1.569 {built-in method builtins.exec}
        1    0.013    0.013    1.569    1.569 <string>:1(<module>)
        1    0.002    0.002    1.556    1.556 <string>:1(<listcomp>)
     1000    1.388    0.001    1.388    0.001 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.002    0.000    0.166    0.000 fromnumeric.py:1710(sum)
     1000    0.000    0.000    0.164    0.000 _methods.py:31(_sum)
     1000    0.164    0.000    0.164    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1000    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [25]:
cProfile.run('[unstenciled_sum(np.random.randint(100, size=(6,10,10)), axes=np.array([0]), checks=False) for i in range(1000)]', sort = 'cumtime')

         75004 function calls in 0.164 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.164    0.164 {built-in method builtins.exec}
        1    0.000    0.000    0.164    0.164 <string>:1(<module>)
        1    0.002    0.002    0.164    0.164 <string>:1(<listcomp>)
     1000    0.005    0.000    0.150    0.000 <ipython-input-19-cc9e41348ac7>:1(unstenciled_sum)
     1000    0.046    0.000    0.131    0.000 <ipython-input-15-15f28e7aa83f>:47(stenciled_sum)
     7000    0.035    0.000    0.038    0.000 <ipython-input-15-15f28e7aa83f>:1(subarray_view)
     4000    0.012    0.000    0.012    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1000    0.010    0.000    0.010    0.000 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.001    0.000    0.010    0.000 shape_base.py:236(hstack)
     6000    0.010    0.000    0.010    0.000 <ipython-input-15-15f28e7aa83f>:26(subrang

In [26]:
cProfile.run('[np.sum(np.random.randint(100, size=(6,10,10)), axis=(0)) for i in range(1000)]', sort = 'cumtime')

         5004 function calls in 0.013 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.013    0.013 {built-in method builtins.exec}
        1    0.000    0.000    0.013    0.013 <string>:1(<module>)
        1    0.001    0.001    0.013    0.013 <string>:1(<listcomp>)
     1000    0.007    0.000    0.007    0.000 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.001    0.000    0.005    0.000 fromnumeric.py:1710(sum)
     1000    0.000    0.000    0.003    0.000 _methods.py:31(_sum)
     1000    0.003    0.000    0.003    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1000    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [27]:
def check_axes_access(axes, array_ndim):
    if np.max(axes) >= array_ndim or np.min(axes)< -array_ndim:
            raise IndexError('too many indices for array')

def convert_axes_to_positive(axes, array_ndim):
    for index, element in enumerate(axes):
            if element < 0:
                axes[index] = element + array_ndim

def check_stencil_shape(array_ndim, axes, summed_axes_shape, stencil):
    correct_stencil_shape = np.hstack( [np.array(summed_axes_shape), np.array(array_ndim - len(axes))] )
    if not np.all(np.array(stencil.shape) == correct_stencil_shape):
            raise ValueError('The shape of the stencil must match the big array and axes appropriately')

def subarray_multislice(array_ndim, axes, indices):
    indices = np.array(indices)
    colon = slice(None, None, None)
    multislice = ()
    for i in range(array_ndim):
        if i in axes:
            multislice = multislice + (indices[np.where(axes==i)[0][0]],)
        else:
            multislice = multislice + (colon,)
    return multislice

class fixed_stencil_sum(object):
    
    def __init__(self, array_ndim, axes_summed_over, summed_axes_shape, stencil):
        axes = np.atleast_1d(np.array(axes_summed_over)).flatten()
        #check that inputs are compatible
        check_axes_access(axes, array_ndim)
        convert_axes_to_positive(axes, array_ndim)
        check_stencil_shape(array_ndim, axes, summed_axes_shape, stencil)
        
        self.array_ndim = array_ndim
        self.axes = axes
        self.not_axes = [i for i in range(array_ndim) if i not in axes]
        self.summed_axes_shape = summed_axes_shape
        #left zero the stencil, ablsa is a tuple of indices into "all but last stencil axis"
        ablsa = tuple(range(stencil.ndim-1))
        stencil = stencil - np.amin(stencil, axis=ablsa)
        self.stencil = stencil
        self.input_expand = np.amax(stencil, axis=ablsa) - np.amin(stencil, axis=ablsa)
        self.iter_bounds = stencil.shape[:-1]
        final_loop = np.product(np.array(self.iter_bounds))
        
        self.stencil_loop_indices = [int_to_indices(i, self.iter_bounds) for i in range(final_loop)]
        self.array_subview_multislices = [subarray_multislice(self.array_ndim, self.axes, indices)
                                         for indices in self.stencil_loop_indices]

    def stenciled_sum(self, big_array):
        subarray_shape = np.array(big_array.shape)[self.not_axes]
        return_array_shape = subarray_shape + self.input_expand
        return_array = np.zeros(return_array_shape, dtype = big_array.dtype)
        for indices, multislice in zip(self.stencil_loop_indices, self.array_subview_multislices):
            starts = self.stencil[indices]
            ends = self.stencil[indices] + subarray_shape
            chunk_to_increase = subrange_view(return_array, starts, ends, checks=False)
            chunk_to_increase[:] += big_array[multislice]
        return return_array

In [28]:
matrix3d_axis0_sum = fixed_stencil_sum(3, 0, 6, np.zeros((6,2), dtype='int_'))

In [29]:
matrix3d_axis0_sum.stencil_loop_indices

[(0,), (1,), (2,), (3,), (4,), (5,)]

In [30]:
matrix3d_axis0_sum.array_subview_multislices

[(0, slice(None, None, None), slice(None, None, None)),
 (1, slice(None, None, None), slice(None, None, None)),
 (2, slice(None, None, None), slice(None, None, None)),
 (3, slice(None, None, None), slice(None, None, None)),
 (4, slice(None, None, None), slice(None, None, None)),
 (5, slice(None, None, None), slice(None, None, None))]

In [31]:
matrix3d_axis0_sum.stencil

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [32]:
cProfile.run('[matrix3d_axis0_sum.stenciled_sum(np.random.randint(100, size=(6,10,10))) for i in range(1000)]', sort = 'cumtime')

         10004 function calls in 0.058 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.058    0.058 {built-in method builtins.exec}
        1    0.000    0.000    0.058    0.058 <string>:1(<module>)
        1    0.001    0.001    0.058    0.058 <string>:1(<listcomp>)
     1000    0.028    0.000    0.045    0.000 <ipython-input-27-1364f4f0d3f6>:51(stenciled_sum)
     6000    0.013    0.000    0.013    0.000 <ipython-input-15-15f28e7aa83f>:26(subrange_view)
     1000    0.011    0.000    0.011    0.000 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.array}
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.zeros}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [33]:
matrix3d_axis0_l5_sum = fixed_stencil_sum(3, 0, 5, np.zeros((5,2), dtype='int_'))

In [34]:
cProfile.run('[matrix3d_axis0_l5_sum.stenciled_sum(np.random.randint(100, size=(5,200,200))) for i in range(1000)]', sort = 'cumtime')

         9004 function calls in 1.632 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.632    1.632 {built-in method builtins.exec}
        1    0.013    0.013    1.632    1.632 <string>:1(<module>)
        1    0.002    0.002    1.619    1.619 <string>:1(<listcomp>)
     1000    1.419    0.001    1.419    0.001 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.124    0.000    0.199    0.000 <ipython-input-27-1364f4f0d3f6>:51(stenciled_sum)
     1000    0.063    0.000    0.063    0.000 {built-in method numpy.core.multiarray.zeros}
     5000    0.009    0.000    0.009    0.000 <ipython-input-15-15f28e7aa83f>:26(subrange_view)
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [35]:
matrix3d_axis0_l100_sum = fixed_stencil_sum(3, 0, 100, np.zeros((100,2), dtype='int_'))

In [36]:
cProfile.run('[matrix3d_axis0_l5_sum.stenciled_sum(np.random.randint(100, size=(100,10,10))) for i in range(1000)]', sort = 'cumtime')

         9004 function calls in 0.129 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.129    0.129 {built-in method builtins.exec}
        1    0.000    0.000    0.129    0.129 <string>:1(<module>)
        1    0.002    0.002    0.128    0.128 <string>:1(<listcomp>)
     1000    0.093    0.000    0.093    0.000 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.021    0.000    0.034    0.000 <ipython-input-27-1364f4f0d3f6>:51(stenciled_sum)
     5000    0.009    0.000    0.009    0.000 <ipython-input-15-15f28e7aa83f>:26(subrange_view)
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.array}
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.zeros}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [37]:
cProfile.run('[unstenciled_sum(np.random.randint(100, size=(100,10,10)), axes=np.array([0]), checks=False) for i in range(1000)]', sort = 'cumtime')

         545004 function calls in 1.349 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.349    1.349 {built-in method builtins.exec}
        1    0.000    0.000    1.349    1.349 <string>:1(<module>)
        1    0.002    0.002    1.349    1.349 <string>:1(<listcomp>)
     1000    0.006    0.000    1.270    0.001 <ipython-input-19-cc9e41348ac7>:1(unstenciled_sum)
     1000    0.475    0.000    1.249    0.001 <ipython-input-15-15f28e7aa83f>:47(stenciled_sum)
   101000    0.457    0.000    0.491    0.000 <ipython-input-15-15f28e7aa83f>:1(subarray_view)
   100000    0.151    0.000    0.151    0.000 <ipython-input-15-15f28e7aa83f>:26(subrange_view)
   100000    0.089    0.000    0.098    0.000 <ipython-input-13-1ea399f969ff>:16(multibase_increment_fast)
     1000    0.075    0.000    0.075    0.000 {method 'randint' of 'mtrand.RandomState' objects}
   101000    0.034    0.000    0.034    0.000

In [38]:
cProfile.run('[np.sum(np.random.randint(100, size=(100,10,10)), axis=(0)) for i in range(1000)]', sort = 'cumtime')

         5004 function calls in 0.087 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.087    0.087 {built-in method builtins.exec}
        1    0.000    0.000    0.087    0.087 <string>:1(<module>)
        1    0.001    0.001    0.086    0.086 <string>:1(<listcomp>)
     1000    0.074    0.000    0.074    0.000 {method 'randint' of 'mtrand.RandomState' objects}
     1000    0.001    0.000    0.012    0.000 fromnumeric.py:1710(sum)
     1000    0.000    0.000    0.010    0.000 _methods.py:31(_sum)
     1000    0.010    0.000    0.010    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1000    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


